### Problem 2: Dot Product

csc596: Accelerate! GPU
<br>
university of miami
<br>
fall 2022-2023 (231)
<br>
Burton Rosenberg
<br>
<br>
<br>last update: Aug 30, 2022

In [2]:
#
# summing elements in an array
# Accelerate! GPU course
#
# last-update:
#     30 aug 2022 -bjr: created
#
#


def total_sum_seq(a):
    s = 0
    for i in range(len(a)):
        s += a[i]
    a[0] = s
    return


# this is a simulation of a GPU algorithm; 
# the phases are sequential, and should synchronize
# the threads are parallel 

# properties: 
#    threads used: n/2^i in phase i
#    memory accesss: fully independent

def total_sum_folding(a): 

    def above_power_2(x):
        i = 1
        while i<x:
            i *= 2
        return i

    l_a = above_power_2(len(a))
    
    phase = 0
    # the first fold takes care of arrays not a power of two
    d = l_a//2
    for thread in range(d):
        if (thread+d)<len(a):
            a[thread] += a[thread+d]

    # remaining phasess do not need the length check
    d = d//2
    while d>0:
        phase += 1 # just a notation;
        
        for thread in range(d):
            a[thread] += a[thread+d]
        d = d//2
            
    return

# it is also possible to deal with non-perfect powers of two as follows

def total_sum_folding_alt(a):
    n = len(a)
    while n>1:
        if n%2==0:
            # even
            d = n//2 
            
            for thread in range(d):
                a[thread] += a[thread+d]
        
            n = d
        else:
            # odd
            d = n//2 # trucation
            
            for thread in range(d):
                a[thread] += a[thread+d+1]
                
            n = d+1
    return




def testing_total_sum(a,b):

    def fill_array(a):
        for i in range(len(a)):
            a[i] = i
        return a

    def testing_total_sum_aux(n):
        a = [0]*n
        
        fill_array(a)
        total_sum_seq(a)
        s_1 = a[0]
        
        fill_array(a)
        total_sum_folding(a)
        s_2 = a[0]
        
        fill_array(a)
        total_sum_folding_alt(a)
        s_3 = a[0]
        
        if s_1!=s_2:
            return False
        if s_1!=s_3:
            return False
        return True

    print(f'Testing total sum between {a} and {b} ...')
    for s in range(a,b):
        if not testing_total_sum_aux(s):
            print('** error')
            return False
    print('** success')
    print()
    return True

testing_total_sum(1000,1050)


Testing total sum between 1000 and 1050 ...
** success



True

### END